<a href="https://colab.research.google.com/github/gladcolor/tweet_classification/blob/master/tweets_suicide_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import modules

In [2]:
import re
import os
import sys

import pandas as pd
from IPython.display import HTML, display

# import emoji
# 
# from simpletransformers.classification import ClassificationModel
import pandas as pd
import smart_open
import numpy as np 
from sklearn import preprocessing


# from simpletransformers.classification import ClassificationModel, ClassificationArgs

pd.set_option('display.max_colwidth', 0)

# Load training dataset of 8400 tweets

In [40]:
pd.set_option('display.max_colwidth', 0)

all_sample_file = r'https://raw.githubusercontent.com/gladcolor/tweet_classification/master/samples_8400_demojized.zip'
samples_df = pd.read_csv(all_sample_file)

# remove emojis
#samples_df['message'] = samples_df['message'].apply(emoji.demojize)

samples_df

,message,Label
0,"do you know if you text home to six hundred and eighty-six thousand, eight hundred and sixty-eight when you be feel down depress or suicidal a crisis worker will text you back immediately and will continue to text with you many people do not feel comfortable <url>",0
1,yes dollhouse sense8 galavant glee white collar nikita firefly dead like me baccano better off ted the sarah connor chronicle battlestar galactica fringe the lie game merlin kyle xy joan of arcadia the cape revolution the shannara chronicle warehouse thirteen,0
2,shit eaters always want people to join them people who should be dead always tell people to kill themselves revel in the happy tweet,0
3,my crush cause me so much emotional pain it be not even funny <url>,0
4,while gov gina raimondo d be not especially enthusiastic about the prospect she tell the publics radio on thursday that she be more open to it than she ever have be <hashtag> <hashtag> <hashtag> <url>,0
...,...,...
8395,nigga really wanna die in the night time,0
8396,i want to die so,1
8397,god adding depression and overthinking when he made me,0
8398,just willing to kill myself every day shout out 2 rey,1


# Create a simple classifier. 

This classifier will label a tweet as positive if it contains the following keywords and have more other 40 characteristics:

1. `i want to die`
2. `i wanna die`
3. `kill myself`
4. `ll kill myself`

The precision and recall are calculated in the end.

In [88]:
def get_positive(samples_df):

    max_extra_char = 40

    samples_df['len'] = samples_df['message'].str.len()

    idx = samples_df['message'].str.lower().str.contains('ould')
    samples_df = samples_df[~idx]

    posi_df_list = []
    # find "i want to die"
    tag_str = "i want to die"
    idx = samples_df['len'] < (len(tag_str) + max_extra_char)
    df = samples_df[idx]
    idx  = df['message'].str.lower().str.contains(tag_str)
    df = df[idx]
    posi_df_list.append(df.copy())

    # # find "i wanna die"
    tag_str = "i wanna die"
    idx = samples_df['len'] < (len(tag_str) + max_extra_char)
    df = samples_df[idx]
    idx  = df['message'].str.lower().str.contains(tag_str)
    df = df[idx]
    posi_df_list.append(df.copy())


    tag_str = "kill myself"
    idx = samples_df['len'] < (len(tag_str) + max_extra_char)
    df = samples_df[idx]
    idx  = df['message'].str.lower().str.contains(tag_str)
    df = df[idx]
    posi_df_list.append(df.copy())

    tag_str = "ll kill myself"
    idx = samples_df['len'] < (len(tag_str) + max_extra_char)
    df = samples_df[idx]
    idx  = df['message'].str.lower().str.contains(tag_str)
    df = df[idx]
    posi_df_list.append(df.copy())


    df_concated = pd.concat(posi_df_list)

    return df_concated


samples_df = pd.read_csv(all_sample_file)
positives = get_positive(samples_df)



precision = positives['Label'].sum() / len(positives)
recall = positives['Label'].sum() / 983

f1_score = 2 * (precision * recall) / (precision + recall)

print("precision: ", precision)
print("recall: ", recall)
print("f1_score: ", f1_score)


positives.sample(40)

precision:  0.5991861648016277
recall:  0.5991861648016277
f1_score:  0.5991861648016277


,message,Label,len
8396,i want to die so,1,16
7305,yeah no i will kill myself before corona does,0,45
1912,@Erick_122333 I want to die,1,27
7241,these mfs forced me here i wanna die,0,36
7175,if anything happens to carti i will kill myself,0,47
7147,i’m going to kill myself tonight it’s definite,1,46
7822,this how i wanna die,0,20
6664,i wanna kill myself,1,19
8288,i feel like i wanna die...,1,26
7316,i’m literally gonna kill myself,1,31


Clean tweets

Similar results

In [86]:
def get_positive(samples_df):

    max_extra_char = 40

    samples_df['len'] = samples_df['message'].str.len()

    idx = samples_df['message'].str.lower().str.contains('ould')
    samples_df = samples_df[~idx]

    posi_df_list = []
    # find "i want to die"
    tag_str = "i want to die"
    idx = samples_df['len'] < (len(tag_str) + max_extra_char)
    df = samples_df[idx]
    idx  = df['message'].str.lower().str.contains(tag_str)
    df = df[idx]
    posi_df_list.append(df.copy())

    # # find "i wanna die"
    tag_str = "i wanna die"
    idx = samples_df['len'] < (len(tag_str) + max_extra_char)
    df = samples_df[idx]
    idx  = df['message'].str.lower().str.contains(tag_str)
    df = df[idx]
    posi_df_list.append(df.copy())


    tag_str = "kill myself"
    idx = samples_df['len'] < (len(tag_str) + max_extra_char)
    df = samples_df[idx]
    idx  = df['message'].str.lower().str.contains(tag_str)
    df = df[idx]
    posi_df_list.append(df.copy())

    tag_str = "ll kill myself"
    idx = samples_df['len'] < (len(tag_str) + max_extra_char)
    df = samples_df[idx]
    idx  = df['message'].str.lower().str.contains(tag_str)
    df = df[idx]
    posi_df_list.append(df.copy())


    df_concated = pd.concat(posi_df_list)

    return df_concated

def clean_twts(tw):
  # remove urls
  pattern = 'https{0,1}:\/\/t.co\/[a-zA-Z0-9]+'
  tw = re.sub(pattern, "", tw)

  # remove @
  pattern = '@[a-zA-Z0-9_]+ '
  tw = re.sub(pattern, "", tw)

  #tw = remove_emoji(tw)

  return tw

def clean_twt_row(row):
  return clean_twts(row['message'])


samples_df = pd.read_csv(all_sample_file)
samples_df['message'] = samples_df.apply(clean_twt_row, axis=1)

positives = get_positive(samples_df)

precision = positives['Label'].sum() / len(positives)
recall = positives['Label'].sum() / 983

f1_score = 2 * (precision * recall) / (precision + recall)

print("precision: ", precision)
print("recall: ", recall)
print("f1_score: ", f1_score)

positives.sample(20)




precision:  0.5793036750483559
recall:  0.6093591047812817
f1_score:  0.5939514129895884


,message,Label,len
1920,Trying to kill myself :sparkles:,1,32
1621,i wanna kill myself,1,19
8066,i’m finna kill myself,1,21
6788,honestly i just wanna kill myself most of the time,1,50
8114,i read this and now i want to die:relieved_face:,0,48
8317,i wanna die,1,11
7264,omo in this life; i can’t kill myself,0,37
3821,I want to die :loudly_crying_face:,1,34
6738,if he keeps whistling i will kill myself,0,40
6799,i’d kill myself :loudly_crying_face:,1,36
